In [1]:
import numpy as np

In [2]:
with open("test.txt") as f:
    test = f.read().splitlines()

In [3]:
test[0]

'Sensor at x=2, y=18: closest beacon is at x=-2, y=15'

In [4]:
xys = [coord for coord in test[0].split() if "=" in coord]
xys

['x=2,', 'y=18:', 'x=-2,', 'y=15']

Account for ',' and ':'

In [5]:
xys[0].strip(',')

'x=2'

In [6]:
import re

In [7]:
p = re.compile('[a-z]+')
m = p.match('tempo')
m.group()

'tempo'

In [8]:
p = re.compile('-?\d+')
p.findall(test[0])

['2', '18', '-2', '15']

In [9]:
re.findall('-?\d+', test[0])

['2', '18', '-2', '15']

In [10]:
xys = [list(map(int, p.findall(line))) for line in test]

In [11]:
xys

[[2, 18, -2, 15],
 [9, 16, 10, 16],
 [13, 2, 15, 3],
 [12, 14, 10, 16],
 [10, 20, 10, 16],
 [14, 17, 10, 16],
 [8, 7, 2, 10],
 [2, 0, 2, 10],
 [0, 11, 2, 10],
 [20, 14, 25, 17],
 [17, 20, 21, 22],
 [16, 7, 15, 3],
 [14, 3, 15, 3],
 [20, 1, 15, 3]]

Convert to complex, since we're only interested in manhattan distance

In [12]:
make_cmplx = lambda sx, sy, bx, by: [complex(sx, sy), complex(bx, by)]

In [13]:
xys_ij = [make_cmplx(*xy) for xy in xys]
xys_ij

[[(2+18j), (-2+15j)],
 [(9+16j), (10+16j)],
 [(13+2j), (15+3j)],
 [(12+14j), (10+16j)],
 [(10+20j), (10+16j)],
 [(14+17j), (10+16j)],
 [(8+7j), (2+10j)],
 [(2+0j), (2+10j)],
 [11j, (2+10j)],
 [(20+14j), (25+17j)],
 [(17+20j), (21+22j)],
 [(16+7j), (15+3j)],
 [(14+3j), (15+3j)],
 [(20+1j), (15+3j)]]

In [14]:
s, b = xys_ij[0]
print(s, b)

(2+18j) (-2+15j)


In [15]:
b-s

(-4-3j)

In [16]:
calc_md = lambda s, b: int(abs((b-s).real)+abs((b-s).imag))

In [17]:
calc_md(*xys_ij[0])

7

In [18]:
mds = [calc_md(*sb) for sb in xys_ij]
mds

[7, 1, 3, 4, 4, 5, 9, 10, 3, 8, 6, 5, 1, 7]

In [19]:
list(zip(xys_ij, mds))

[([(2+18j), (-2+15j)], 7),
 ([(9+16j), (10+16j)], 1),
 ([(13+2j), (15+3j)], 3),
 ([(12+14j), (10+16j)], 4),
 ([(10+20j), (10+16j)], 4),
 ([(14+17j), (10+16j)], 5),
 ([(8+7j), (2+10j)], 9),
 ([(2+0j), (2+10j)], 10),
 ([11j, (2+10j)], 3),
 ([(20+14j), (25+17j)], 8),
 ([(17+20j), (21+22j)], 6),
 ([(16+7j), (15+3j)], 5),
 ([(14+3j), (15+3j)], 1),
 ([(20+1j), (15+3j)], 7)]

Given a point and the manhattan distance, `d_m`, find all points within this radius

Wrong approach

Given a sensor, its reach, and the target row, find the extent of the sensor's detection cone on that row

E.g. 

- sensor at (2, 2)
- closest beacon (6, 5)
- extent of reach on row 4?

1. Find d_m = 6 - 2 + 5 - 2 = 7
1. On row = sx, reach ranges from 2 - 7 = -5 to 2 + 7 = 9
1. on row = 4, `d_new = d - abs(ty - sy) = 7 - abs(4 - 2) = 5`
1. reach extends -5/+5 from sx: 2 - 5 = -3, 2 + 5 = 7
1. Thus all nodes from (-3, 4) to (7, 4) cannot contain a beacon (`True`)

In [32]:
def calc_reach(s: complex, reach: int, row: int):
    r_new = reach - abs(row - s.imag)
    if r_new > 0:
        left = (s.real - r_new)
        return left, r_new*2
    else:
        return None

In [37]:
row = 10
reaches = [d for xy, reach in zip(xys_ij, mds) if (d := calc_reach(xy[0], reach, row))]
reaches

[(2.0, 12.0), (-2.0, 4.0), (16.0, 8.0), (14.0, 4.0)]

So we've got these ranges:

- 2 + 12 to 14
- -2 + 4 to 2
- 16 + 8 to 24
- 14 + 4 to 18

From these, we can determine everything from -2 to 24 cannot contain a beacon: 27 nodes; however one of those nodes do contain a beacon, and so only the remaining 26 nodes cannot

Two problems:

- how to discern overlaps?
- how to discern whether beacons are located in any one of the nodes?

Overlaps:

1. Sort left edge
1. Consider next left edge
  - If left_new <= right_prev:
    - overlap = right_prev - left_current + 1 (if equal, there is overlap of 1 node)
    - right_current = right_prev + reach_current - overlap
    - e.g. A: -2 + 4 to 2; B: 2 + 12 to 14
    - 2 <= 2
    - overlap = 2 - 2 + 1
    - right edge = right_current = 14; left edge = left_prev = -2
  - else, if left_new > right_prev:
    - update total = right_prev - left_prev + 1 (inclusive)
    - Check for beacons that are on the same row (y=2000000)
    - for each beacon on the specified row:
      - check if `left_prev <= x_beacon <= right_prev`
      - if yes, pop beacon from pool, increment update
      - else, continue to next beacon
    - update left_prev = left_new
    - update right_prev = left_prev + reach_new


In [38]:
reach_sorted = sorted(reaches, key=lambda x: x[0])

In [39]:
reach_sorted

[(-2.0, 4.0), (2.0, 12.0), (14.0, 4.0), (16.0, 8.0)]

In [40]:
for (edge, reach) in reach_sorted:
    print(f"edge: {edge}, reach: {reach}")

edge: -2.0, reach: 4.0
edge: 2.0, reach: 12.0
edge: 14.0, reach: 4.0
edge: 16.0, reach: 8.0


In [31]:
b_search = [int(b[1].real) for b in xys_ij if b[1].imag == 10]
b_search = list(set(b_search))

In [71]:
def beacon_elim(target: int, reach: list, sb_pairs: list = None):
    """
    Given a sorted list of left edges and their reach, find
    how many nodes cannot contain a beacon
    
    Parameters
    --------
    target: int
        Beacon elimination will be conducted for this row
    reach: list(tuple)
        each tuple contains the left edge and the reach of the sensor detection
        cone
    sb_pairs: list
        list of tuples containing location of sensors and beacons
        in complex coordinates
    
    Returns
    -------
    num_empty: int
        number of nodes on the target row that cannot contain beacon
    """
    # sort by left edge
    reach_sorted = sorted(reaches, key=lambda x: x[0])
    b_search = [int(b[1].real) for b in sb_pairs if b[1].imag == target]
    b_search = list(set(b_search))
    
    
    left_prev, reach_prev = reach_sorted[0]
    right_edge = left_prev + reach_prev
    totals = [reach_prev]
    print(f"left: {left_prev}, reach: {reach_prev}, right: {right_edge}")
    # reach_sorted.remove((left_prev, reach_prev))
    for (left_edge, reach_new) in reach_sorted[1:]:
        print(f"left: {left_edge}, reach: {reach_new}, right: {right_edge}")
        if left_edge <= right_edge:
            # update right edge only
            right_edge = left_edge + reach_new
            # update last subtotal accounting for overlap
            totals[-1] = right_edge - left_prev
            print(f"overlap - new right: {right_edge}")
            
        else:
            # new segment
            totals.append(reach_new)
            print(f"new segment: {reach_new} added to total")
            # update the edges
            left_prev = left_edge
            right_edge = left_prev + reach_new
        
        
        # check for beacons
        for b in b_search:
            if left_edge <= b and b <= right_edge:
                totals[-1] -= 1
                print(f"beacon found within range at node {b}")
            
                    
    return sum(totals)
        

In [72]:
beacon_elim(10, reaches, xys_ij)

left: -2.0, reach: 4.0, right: 2.0
left: 2.0, reach: 12.0, right: 2.0
overlap - new right: 14.0
beacon found within range at node 2
left: 14.0, reach: 4.0, right: 14.0
overlap - new right: 18.0
left: 16.0, reach: 8.0, right: 18.0
overlap - new right: 24.0


26.0

In [57]:
foo = [(1,2),(2,3),(3,4)]
list(reversed(foo)).pop()

(1, 2)

In [1]:
baa = set([1,2,3,3])
baa.__dir__()

['__new__',
 '__repr__',
 '__hash__',
 '__getattribute__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__iter__',
 '__init__',
 '__sub__',
 '__rsub__',
 '__and__',
 '__rand__',
 '__xor__',
 '__rxor__',
 '__or__',
 '__ror__',
 '__isub__',
 '__iand__',
 '__ixor__',
 '__ior__',
 '__len__',
 '__contains__',
 'add',
 'clear',
 'copy',
 'discard',
 'difference',
 'difference_update',
 'intersection',
 'intersection_update',
 'isdisjoint',
 'issubset',
 'issuperset',
 'pop',
 '__reduce__',
 'remove',
 '__sizeof__',
 'symmetric_difference',
 'symmetric_difference_update',
 'union',
 'update',
 '__class_getitem__',
 '__doc__',
 '__str__',
 '__setattr__',
 '__delattr__',
 '__reduce_ex__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__dir__',
 '__class__']

In [3]:
for i in baa.copy():
    if i % 2 == 0:
        baa.discard(i)

In [5]:
baa | {1,2}

{1, 2, 3}

In [7]:
foo = list(range(4))
for i, val in enumerate(foo):
    if val > 1:
        foo[i] = 4
foo

[0, 1, 4, 4]